# Classifiez automatiquement des biens de consommation - partie textuelle - modélisation non supervisée

## 1. Introduction

__Sommaire de tous les notebooks de ce dossier__

1. Analyse exploratoire des données textuelles
2. Analyse exploratoire des données visuelles__
3. __Prétraitement, feature extraction et faisabilité textuelle - premiers modèles__  
4. Prétraitement, feature extraction et faisabilité visuelle - premiers modèles

Dans ce notebook, nous allons traiter du troisième point. 

### Les critères à remplir dans ce notebook
   1. __Prétraiter des données textes pour obtenir un jeu de données exploitable.__
      - [ ] CE1 Vous avez nettoyé les champs de texte (suppression de la ponctuation et des mots de liaison, mise en minuscules)
      - [x] CE2 Vous avez écrit une fonction permettant de “tokeniser” une phrase.
      - [x] CE3 Vous avez écrit une fonction permettant de “stemmer” une phrase.
      - [x] CE4 Vous avez écrit une fonction permettant de “lemmatiser” une phrase.
      - [ ] CE5 Vous avez construit des features ("feature engineering") de type bag-of-words (bag-of-words standard : comptage de mots, et Tf-idf), avec des étapes de nettoyage supplémentaires : seuil de fréquence des mots, normalisation des mots.
      - [ ] CE6 Vous avez testé une phrase ou un court texte d'exemple, pour illustrer la bonne réalisation des 5 étapes précédentes.
      - [ ] CE7 Vous avez, en complément de la démarche de type “bag-of-words”, mis en oeuvre 3 démarches de word/sentence embedding : Word2Vec (ou Doc2Vec ou Glove ou FastText), BERT, et USE (Universal Sentence Encoder).
      - [ ] CE8 Vous vous êtes assurés que le texte traité ne relève pas d’une propriété intellectuelle dont l’utilisation ou la modification est interdite.

   2. __Mettre en œuvre des techniques de réduction de dimension.__
      - [ ] CE1 Vous avez justifié la nécessité de la réduction de dimension. 
      - [ ] CE2 Vous avez appliqué une méthode de réduction de dimension adaptée à la problématique (ex. : ACP). 
      - [ ] CE3 Vous avez justifié le choix des valeurs des paramètres dans la méthode de réduction de dimension retenue (ex. : le nombre de dimensions conservées pour l'ACP).  
 
   3. __Représenter graphiquement des données à grandes dimensions.__
      - [ ] CE1 Vous avez mis en œuvre au moins une technique de réduction de dimension (via LDA, ACP, T-SNE, UMAP ou autre technique).
      - [ ] CE2 Vous avez réalisé au moins un graphique représentant les données réduites en 2D (par exemple affichage des 2 composantes du T-SNE).
      - [ ] CE3 Vous avez réalisé et formalisé une analyse du graphique en 2D.

## Préparation de l'environnement

__Importation des librairies et du jeu de données__

In [2]:
# Installation des librairies utiles pour la partie textuelle
#!pip3 install --quiet "tensorflow-text==2.8.*" # Mettre le bon chiffre final
#!pip3 install --quiet "tensorflow-text
#!pip install umap-learn
#!pip install tensorflow==2.8.4
#!pip install nltk
#!pip install enchant #pour les fautes d'orthographe

# Importation de librairies de base
from google.colab import drive
import sys
import logging
import os
from os import listdir
os.environ["TF_KERAS"]='1'

import pandas as pd
import numpy as np
import time
from IPython.display import display

# Librairie pour la visualisation 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib.colors import ListedColormap, BoundaryNorm
import plotly.express as px

# Nettoyage textuelle
import string
import re
import unicodedata
import random
from scipy.stats import randint as sp_randint
#import enchant

# Tokenisation, Stemmatisation et Lemmatisation 
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
#nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import Input, GlobalAveragePooling1D, Dropout, Embedding, Dense
from keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.regularizers import L2
from tensorflow.keras.optimizers import Adam
import multiprocessing
#import torch
import transformers
from transformers import AutoTokenizer, TFAutoModel

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import RandomizedSearchCV
# librairies pour l'extraction des features
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize, FunctionTransformer, LabelEncoder

# librairies pour la réduction des dimensions
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation, PCA

# librairies pour le clustering
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, SpectralClustering, Birch
from sklearn.mixture import GaussianMixture

# librairie pour l'évaluation du modèle 
from sklearn.metrics import (homogeneity_score, v_measure_score, completeness_score, 
                             confusion_matrix, precision_score, recall_score, 
                             accuracy_score, adjusted_rand_score, classification_report)
from joblib import dump, load

In [3]:
# Initialisation du random seed à 42
#np.random.seed(42)
#from sklearn.utils import check_random_state
#random_state = check_random_state(42)
#random_seed = 42

In [1]:
ls

Terrien_Audrey_1_exploration_textuelle.ipynb
Terrien_Audrey_2_exploration_visuelle.ipynb
Terrien_Audrey_3_classification_non_supervisée_textuelle.ipynb
Terrien_Audrey_4_classification_non_supervisée_ancienne_visuelle.ipynb


In [2]:
cd ../

/Users/audreyterrien/Documents/github_repositories/DS_Master_project_6


In [3]:
import pandas as pd
from my_packages import PATH_input

In [4]:
# Réimportation uniquement des données textuelles nécessaires
df = pd.read_csv(PATH_input+'flipkart_com-ecommerce_sample_1050.csv')    
df['CATEGORY'] = df['product_category_tree'].map(lambda x: x.split("[\"")[1].split(" >>", 1)[0])
df = df[['CATEGORY', 'description']]
df.columns = ['CATEGORY', 'TEXT']
df[df.index==222]

,CATEGORY,TEXT
222,Home Decor & Festive Needs,"Purpledip Kattle Showpiece - 15 cm (Steel, M..."


In [5]:
# Télécharger la liste de stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import word_tokenize
from re import sub
from string import punctuation
import unicodedata

In [6]:
# Les mots à supprimer
unwanted_words = set(['', 'products', 'product', 'free', 'rs', 'buy', 'delivery', 'shipping', 'cash', 'cm', 
                      'flipkart', 'com', 'flipkartcom', 'online', 'price', 'sales', 'features', 'Genuine', 'india',
                      'specifications', 'discounts', 'prices', 'key', 'great'])
flipkart_stopword = list(stop_words)+list(unwanted_words)

In [7]:
# Fonctions de nettoyage et de normalisation du texte // version NLTK 

# Fonctions de nettoyage (CE1)
def cleaning_text(doc, stopword_step=True):
    # Définir une expression régulière pour identifier les points collés aux 
    # mots qui ne sont pas des domaines informatiques (.com, .gov, etc.) 
    # et qui sont dû à des fautes de frappes
    pattern_dot = r"(?<=\w)\.(?=\w)"

    # Définir une expression régulière pour identifier les 's collés aux mots
    pattern_s = r"(?<=\w)'s"

    # Appliquer les substitutions sur le texte
    doc = sub(pattern_dot, " ", doc)
    doc = sub(pattern_s, "", doc)

    # Passage en minuscule
    doc = doc.lower()

    # Suppression des chiffres
    doc = ''.join([ch for ch in doc if not ch.isdigit()])
    
    # Suppression de la ponctuation
    doc = doc.translate(str.maketrans('', '', punctuation))
    
    # Suppression des accents
    doc = unicodedata.normalize('NFKD', doc).encode('ASCII', 'ignore').decode('utf-8')
    
    # Choix de suppression des mots communs ou de liaisons 
    if stopword_step == True:
      # Suppression des mots de liaison
      #stop_words = set(stopwords.words('english'))
      words = doc.split()
      words = [word for word in words if word not in flipkart_stopword]
      doc = ' '.join(words)

    # Correction de fautes d'orthographe hypersimplifié utilisant la librairie
    # enchant
    # doc = correct_doc(doc) -> pas assez de connaissance 
    # pour être sûr de l'utiliser correctement

    # Suppression des espaces et des retours à la ligne
    doc = sub(r'\s+', ' ', doc).strip()
    return doc

# fonction qui nettoie et tokénise un texte (CE2)
def tokenize(doc):
    doc = cleaning_text(doc, stopword_step=True)
    tokens = word_tokenize(doc)
    tokens = [token for token in tokens if len(token) > 2]
    return tokens

# fonction qui nettoie et tokénise pour un texte traité par une technique dl (CE2)
def tokenize_dl(doc):
    doc = cleaning_text(doc, stopword_step=False)
    tokens = word_tokenize(doc)
    return tokens

# Fonction de stemmatisation (CE3) 
stemmer = PorterStemmer()
def stemming(doc):
    tokens = tokenize(doc)
    stems = [stemmer.stem(token) for token in tokens]
    return stems

# Fonction de lemmatisation (CE4)
lemmatizer = WordNetLemmatizer()
def lemmatisation(doc):
    tokens = tokenize(doc)
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

# Fonction de préparation d'un doc passant par une tokenisation
def transform_bow_fct(doc) :
    tokens = tokenize(doc)  
    doc = ' '.join(tokens)
    return doc

# Fonction de préparation du texte pour le Deep learning (USE et BERT)
def transform_dl_fct(doc) :
    tokens = tokenize_dl(doc)  
    doc = ' '.join(tokens)
    return doc

# Fonction de préparation d'un doc passant par une lemmatisation
def transform_bow_lemm_fct(doc) :
    lemms = lemmatisation(doc)  
    doc = ' '.join(lemms)
    return doc

# Fonction de préparation d'un doc passant par un stemming
def transform_bow_stem_fct(doc) :
    stems = stemming(doc)  
    doc = ' '.join(stems)
    return doc

In [8]:
%%time
# Prétraitement textuels

# tokenisation
df['TOKEN'] = df['TEXT'].map(tokenize)

# stemming
df['STEM'] = df['TEXT'].map(stemming)

# Lemmatisation
df['LEMM'] = df['TEXT'].map(lemmatisation)

# tokenisation + transformation en phrases
df['sentence_bow'] = df['TEXT'].map(transform_bow_fct)

# stemming + transformation en phrases
df['sentence_stem'] = df['TEXT'].map(transform_bow_stem_fct)

# Lemmatisation + transformation en phrases
df['sentence_lemm'] = df['TEXT'].map(transform_bow_lemm_fct)

# tokenisation pour modèle DL + transformation en phrases
df['sentence_dl'] = df['TEXT'].map(transform_dl_fct)

CPU times: user 5.8 s, sys: 75.5 ms, total: 5.88 s
Wall time: 5.93 s


In [9]:
# Taille maximum des features selon les techniques de prétraitement du texte
df['length_bow'] = df['sentence_bow'].apply(lambda x : len(word_tokenize(x)))
df['length_dl'] = df['sentence_dl'].apply(lambda x : len(word_tokenize(x)))
df['length_bow_lemm'] = df['sentence_lemm'].apply(lambda x : len(word_tokenize(x)))
df['length_bow_stem'] = df['sentence_stem'].apply(lambda x : len(word_tokenize(x)))

print("Nombre de mots maximum dans le bag-of-words avec la méthode de tokénisation :", df['length_bow'].max())
print("Nombre de mots maximum dans le bag-of-words avec la méthode de lemmatisation :", df['length_bow_lemm'].max())
print("Nombre de mots maximum dans le bag-of-words avec la méthode de stemming :", df['length_bow_stem'].max())
print("Nombre de mots maximum pour le deep learning (DL) avec la méthode de tokénisation :", df['length_dl'].max())

Nombre de mots maximum dans le bag-of-words avec la méthode de tokénisation : 330
Nombre de mots maximum dans le bag-of-words avec la méthode de lemmatisation : 330
Nombre de mots maximum dans le bag-of-words avec la méthode de stemming : 330
Nombre de mots maximum pour le deep learning (DL) avec la méthode de tokénisation : 562


In [10]:
# Premier exemple pour chaque type de modification (CE6)
row_index = 220
example_df = pd.DataFrame({'0. TEXT': df['TEXT'][row_index], 
                           '1. CLEAN_TEXT': cleaning_text(df['TEXT'][row_index]),
                           '2. TOKENIZATION': str(tokenize(df['TEXT'][row_index])),
                           '2B. TOKENIZED DOC': str(transform_bow_fct(df['TEXT'][row_index])),
                           '3. STEMMING': str(stemming(df['TEXT'][row_index])),
                           '3B. STEMMATIZED DOC': str(transform_bow_stem_fct(df['TEXT'][row_index])),
                           '4. LEMMATIZATION': str(lemmatisation(df['TEXT'][row_index])),
                           '4B. LEMMATIZED DOC': str(transform_bow_lemm_fct(df['TEXT'][row_index])),
                          }, index=['Test on sample 220']).rename_axis('Type de modification').T
example_df

Type de modification,Test on sample 220
0. TEXT,Poppins Printed Baby Boy's Jumpsuit\r\n ...
1. CLEAN_TEXT,poppins printed baby boy jumpsuit high quality...
2. TOKENIZATION,"['poppins', 'printed', 'baby', 'boy', 'jumpsui..."
2B. TOKENIZED DOC,poppins printed baby boy jumpsuit high quality...
3. STEMMING,"['poppin', 'print', 'babi', 'boy', 'jumpsuit',..."
3B. STEMMATIZED DOC,poppin print babi boy jumpsuit high qualiti fu...
4. LEMMATIZATION,"['poppins', 'printed', 'baby', 'boy', 'jumpsui..."
4B. LEMMATIZED DOC,poppins printed baby boy jumpsuit high quality...
